In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
import joblib

from ipynb.fs.full.hog_config import hog_human, WIN_SIZE

In [2]:
datasets = [
        (1, "dataset/human_pos"),
        (0, "dataset/human_neg")
         ]

In [3]:
def crop_img(img, scale=0.8):
    hh = int(img.shape[0]*scale)
    ww = int(img.shape[1]*scale)
    yy = int((img.shape[0]-hh)/2)
    xx = int((img.shape[1]-ww)/2)
    return img[yy:yy+hh, xx:xx+ww]

In [4]:
features = np.zeros((1, 216), np.float32)
labels = np.zeros(1, np.int64)

for ds in datasets:
    label = ds[0]
    path = ds[1]
    image_files = glob.glob(path + "/*")
    for image_file in image_files:
        img = cv2.imread(image_file, cv2.IMREAD_COLOR)
        img = crop_img(img)
        resized_im = cv2.resize(img, WIN_SIZE)
        descriptor = np.transpose(hog_human.compute(resized_im))
        features = np.vstack((features, descriptor))
        labels = np.vstack((labels, label))
features = np.delete(features, (0), axis=0)
labels = np.delete(labels, (0), axis=0).ravel()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    features, 
    labels, 
    test_size=0.2, 
    random_state=42
)

In [6]:
print("== train set ==\n", x_train.shape, y_train.shape)
print('== test set ==\n', x_test.shape, y_test.shape)

clf = svm.SVC(probability=True)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print('=== RESULT ===')
print('Accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

joblib.dump(clf, 'model_hog_svm.pkl') 

== train set ==
 (1783, 216) (1783,)
== test set ==
 (446, 216) (446,)
=== RESULT ===
Accuracy:  0.9461883408071748
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       237
           1       0.95      0.93      0.94       209

    accuracy                           0.95       446
   macro avg       0.95      0.95      0.95       446
weighted avg       0.95      0.95      0.95       446



['model_hog_svm.pkl']